In [148]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk
from sklearn.model_selection import train_test_split
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/vuong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [149]:
from datasets import load_dataset

ds = load_dataset("uitnlp/vietnamese_students_feedback")

In [150]:
ds

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

In [151]:
import pandas as pd
import dataset
# Chuyển tập train từ Hugging Face dataset sang pandas DataFrame
data= ds['train'].to_pandas()
val_df = ds['validation'].to_pandas()
test_df = ds['test'].to_pandas()

In [152]:
data

,sentence,sentiment,topic
0,slide giáo trình đầy đủ .,2,1
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",2,0
2,đi học đầy đủ full điểm chuyên cần .,0,1
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",2,0
...,...,...,...
11421,chỉ vì môn game mà em học hai lần mà không qua...,0,1
11422,em cảm ơn cô nhiều .,2,0
11423,giao bài tập quá nhiều .,0,0
11424,"giáo viên dạy dễ hiểu , nhiệt tình .",2,0


In [153]:


texts = data['sentence'].tolist()
labels = data['sentiment'].tolist()
#labels = data['label'].map({'Positive': 0, 'Negative': 1, 'Neutral':2}).tolist()

In [154]:
from collections import Counter
print(Counter(labels))

Counter({2: 5643, 0: 5325, 1: 458})


In [155]:
nltk.data.find('tokenizers/punkt')  # Kiểm tra xem đã có chưa
nltk.download('punkt', force=True)
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/vuong/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /home/vuong/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [156]:
#mport re
#data = pd.read_csv('code.csv').dropna()
#texts = [re.sub(r"[^\w\s]", "", t.lower()) for t in data['text']]
#labels = data['label'].map({'Positive': 0, 'Negative': 1, 'Neutral': 2}).tolist()

tokenized_texts = [word_tokenize(t) for t in texts]
all_words = [w for txt in tokenized_texts for w in txt]
most_common = Counter(all_words).most_common(4998)

vocab = {'<PAD>': 0, '<UNK>': 1}
for i, (w, _) in enumerate(most_common, 2):
    vocab[w] = i

def to_indices(tokens, max_len):
    idxs = [vocab.get(t, 1) for t in tokens][:max_len]
    return idxs + [0] * (max_len - len(idxs))

max_len_text = 50
text_indices = [to_indices(t, max_len_text) for t in tokenized_texts]

train_texts, test_texts, train_labels, test_labels = train_test_split(text_indices, labels, test_size=0.1, random_state=42)

In [157]:
from classes import SentimentDataset
train_dataset = SentimentDataset(train_texts, train_labels)
test_dataset = SentimentDataset(test_texts, test_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [158]:
data_to_save = {
    'train_loader': train_loader,
    'test_loader': test_loader,
    'batch_size': 16,
    'vocab': vocab,
    'vocab_size': len(vocab)
}
save_path = 'sentiment_data_loader.pth'
torch.save(data_to_save, save_path)

1130